<a href="https://colab.research.google.com/github/dbremont/Notas/blob/main/Papers/Computacion/Kafka%3A_a_Distributed_Messaging_System_for_Log_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Log  processing  has  become  a  critical  component  of  the  data 
pipeline for consumer internet companies. We introduce Kafka, a 
distributed messaging system that we developed for collecting and 
delivering high volumes of log data with low latency. Our system 
incorporates  ideas  from  existing  log  aggregators  and  messaging 
systems,  and  is  suitable  for  both  offline  and  online  message 
consumption.  We  made  quite  a  few  unconventional  yet  practical 
design choices in Kafka to make our system efficient and scalable. 
Our experimental results show that Kafka has superior 
performance  when  compared  to  two  popular  messaging  systems. 
We  have  been  using  Kafka  in  production  for  some  time  and  it  is 
processing hundreds of gigabytes of new data each day.

[Apache Kafka Use Cases: When To Use It & When Not To](https://www.upsolver.com/blog/apache-kafka-use-cases-when-to-use-not)

Distributed messaging system that collect and deliver  high volumes of log data with low latency.

- messaging system?
- low latency?

- What other messaging systems are there?


'Log data':
- user logings,
-  clicks,
- "likes",
- sharing',  
- comments,
- sysetm utilization,
- cpu,
- memory,

I can play with the idea, recolecting metrics, in my computer, of mouse moves, key press, memory use
- And store those as events,


Log data is larger then 'real' data.

Every day 'China Mobile' collects 5-8TB of various user activity events.

Back in the day the system for processing this log information usually scrapce physical log files. Now there are many 'distributed log aggregators' like **Facebook Scribe**, **Yahoo's Data Highway**, **Claudera's Flume**

Kafka is bouth 
- a log agregator,
- a messaging system

Kafka simplifies infraestruc because it is used for offline and online processing of 'logs events'


 

## Related Work

JMS: api, features, garantees.
- Does it alows to batch events in one request?
- What is the distributed support of JMS

 What is the implementation detail leaks by 'Flume'?

Push vs Pull model

Why Kafka uses the pull model?




##  Kafka Architecture and Desigg Principles

**Topic**: A stream of messages of a particular type

**A Producer**: Can produce publish messages to a topic. The published messages are thenn stored at a set of servers called *brokers*.

**A Consumer**: Can subscribe to one or more topics from the brokers, and consume the subscribed messages by pulling data from the brokers.

**A partition**: A way to divide the topics into multiple brokers.

**Brokers**: Are the servers that store the partitions, serverd both producers and consumers.

```java
// Sample of producer code

producer = new Producer(…);
 message = new Message(“test message str”.getBytes());
 set = new MessageSet(message);
 producer.send(“topic1”, set);

```

```java
// Sample of consumer code
 streams[] = Consumer.createMessageStreams(“topic1”, 1)
 for (message : streams[0]) {
 bytes = message.payload();
 // do something with the bytes
 }
```







## Efficiency on a Single Partition

Desing desitions in Kafka:

- Simple Storage: Each partition of a topic correcponds to a logical log. Physically, a log is implemented as a set of segment files of aproximately the same size (e.g 1GB). A message is only exposed to the consumers after it if flushed to disk.

- Mesages are **referred by the offset in disk**, not by  messages id.

- A consumer always consumes messages from a particular partition sequentially. If the consumer acknoledges a particular mesage offset, it implies that the consumer has recieved al messages prio to that offest in the partition. Behind the scences, the consuemr is issuing asynchonous pull reqeusts to the broker to have a buffer of data read for the application to consume.

- Each pull request contains the offset of the message from which the comsumption beings and an acceptable number of bytes to fetch.

- Each broker keeps in memory a softted list of offfsets, includign offsets of the first meesage in every segment file. 

- After a concumer recieves a meesage, it computes the offset of the netxt message to consume and useds it in the next pull request.

**Efficient transfer**

Although the end Consumer api iterates one message at a time, under the covers, each pull req4uest froma  consumer also retrieves multiple messages up to a certain size, typically hundreads of kilobytes.

Another unconventional choice is to avoid explicitely catchign mesages in emory at the Kafka layer. Instead, we rely on the underlying file sytsem page cache. This eliminates double buffering.

Since **Kafka** done do cache at the process level, it makes it feasable to implement it in a Vm-based language, since the garbage collection overhead its recuced by using this technique.

The network access is optimized for consumers. Kafka is a multi-subscriber system and a single message may be consumed multiple times by different consumer applications.

A typical approach to sending bytes froma  local file to a remote socket involves the folowign steps⁉

- 1) read the data from the storage media to the page cache in an OS
- 2) copy data in the page cache to an application buffer
- 3) copy application buffer to another kernell buffer
- 4) send the kernel buffer to the socket 

This gets optimized in linux, because there is a system call *sendfile* that can directly transfer bytes from a file chanell to a socket chaneel.

This avoids the two kernel calls.

**Stateless broker**

The information about how much each consumer has consumed is not maintained by th broker, but by the consumer itself. This makes tricky to delete a message because Kafka dones not know wheter all subscribers have consumed the message. Kafka solves this problem by suigna simple time-based **SLA** for the retention policy. 

A message is automatically deleted if it has been retained in the broker longer than a certain period, typically 7 days. 

A consumer can rewind to and old offset and re-consume data. This biolates the common contract of queue, but proves to be essential feature for many consumers. 

Re-play messages.

Note that rewinding a consumer is much easier to
support in the pull model than the push model.




 ## Distributed Coordination

How the producers and the consumers behave in a distributed setting.

Each producer can publish a message to either a randomly selected partition or a partition semanticaly determined by a partitioning key and a partitioning function. 

How the consumers interact with the brokers?

Kafka has the concept of *consumer groups*. Each consumer group consists of one or more consumers that jointly consume a set of subscribed topics, ie, each mesasge is delivered to only one of the consumers withing the group.

Patition as the smallest unit of parallelism.

Not has a "master" node, but instead let consumers coordinate among themselves in a decentralized fashion. To faciliate the coordination,  we employ a highly aviable consensus service Zookeeper. Zookeeper has a very simple, file system like **API**. One can create a path, set the value of the path read the value of a path, delete a path, and list the children path, ...

Kakfa uses Zookeeper for the followign tasks:
- detecting the addition and the removal of brokers and consumers,
- triggering a rebalance process in each consumer when the above events happend,
- maintaining the consumption relationship and keeping track of th econsumed offset of each partition

Specifically, when each broker or consume starts up, it stores tis information ina broker or consumer registry in Zookeeper. 



## Delivery Guarantees

In general, Kafka only guarantees at-least-once delivery. Exacly-once delivery typically requires two-phase commits.

To avoid log corruption, kafka stores CRC for each message in the log.

## Kafka Usage at LinkedIn

Instrumenting producers

Apache Hadoop

MapReduce

Avro as a serialization protocol because supports schema evolution.

## Experimental Results

## Conclusion and Feture Works

Like other messaging sytems, Kafka employs a pull-based comsumption model that allows an application to consume data at its own rate and rewidn the comsumption whenever needed.  By focusing on log processing paplication, Kafka achieves much higher throughput than conventional messaging systems. It also provided integrate distributed support and can scale out. 

Todo:

- Try to use kafka,
- Try to profile kafka,
- Try to reconsume messages,
- Ankify,
- Readread,
- Try to replicate kafka results.

